## 김포 제주 노선 가격 정보 읽어오기
- 대상 사이트 : 에어부산 웹사이트

### 에어부산 사이트 웹 크롤링
    - 에어부산 국내선 항공권예매 페이지를 이용한 크롤링
    - 기본 크롤링
    - 필요 데이터 추출 하여 Pandas 의 DataFrame 형태로 생성
    - 생성된 데이터 엑셀 파일로 저장

In [35]:
from IPython.display import display
import requests
import pandas as pd
import numpy as np
from pandas import DataFrame
from bs4 import BeautifulSoup
import time
from datetime import datetime
from datetime import timedelta
from common.crawling_util import simple_crawling
from common.parsing_util import parsing_json_data_to_dict,mining_value_by_last_field_name,stat_fare

dom_int='D' ## 국내선(D) 국제선(I) 구분
def crawling_BX_data(dpt,arr,dpt_date):
    ##출발지, 도착지, 출발일을 기준으로 국내선(국제선) 편도 가격 읽어오기
    print('Crawling airbusan homepage schedule site')
    url = "https://www.airbusan.com/web/bookingApi/domesticAvail"
    head = {
        'Referer':'https://www.airbusan.com/web/individual/booking/domestic',
    }
    param ={
    'depDate':dpt_date,
    'depCity':dpt,
    'arrCity':arr,
    'bookingCategory':'Individual',
    'foc':'N',
    'bookingClass':'ES'
    }

    return simple_crawling(url,param,head=head,method='get',json=True)

data_heads = ['date','flt','start','end','fare1','fare2','tax1','tax2','seat']
key_fields = ['depDate','flightNo','depTime','arrTime','fareNetNormal','fareNormal','fuelAD','taxAD','availSeat']
## raw 데이터로 부터 DataFrame 생성
def raw_to_df(raw_data):
    df = pd.DataFrame(mining_value_by_last_field_name(json,key_fields),columns=key_fields)
    df.columns = data_heads
    return df

## 하루 기준 가격정보, 텍스정보, 최소값, 최대값, 평균 DataFrame 생성
def read_BX_1day_fare(dpt,arr,dpt_date):
    ## 데이터 읽어오기
    raw_data = crawling_BX_data(dpt,arr,dpt_date)
    df=raw_to_df(raw_data)
    if df is None or len(df) == 0: ## 읽는 도중 에러가 나거나 익셉션 발생으로 문제가 있을 경우 처리
        print('********** No Data On raw_to_df **********')
        return None

    ## 최소값 최대값 평균 계산
    stat = stat_fare(df,['fare1','fare2'])
    ## 요약정보 추가
    df.ix[len(df)] = [dpt_date,'min','max','mean',str(stat[0]),str(stat[1]),str(stat[2]),'','']
    return df

## 정해진 기간의 데이터 읽어오기, 기본 30일
def read_BX_date_range_fare(dpt,arr,start=0,end=31):
    # 기본 30일간 데이터 읽어서 파일로 저장
    date_range = [ (datetime.today()+timedelta(1)*i).strftime('%Y%m%d') for i in range(start,end)]
    df_list = []
    for d in date_range:
        try:
            fare_df = read_BX_1day_fare(dpt,arr,d)
            if fare_df is not None:
                df_list.append(fare_df)
        except Exception as e:
            print('****** Error occured : ',e)
    result = pd.concat(df_list,ignore_index=True)
    print('++++++++++Total : ', len(result))
    ## 파일 저장
    result.to_excel('{}/{}_{}_{}_{}_{}_{}.xls'.format('excel','BX',dpt,arr,start,end,datetime.today().strftime('%Y%m%d%H%M')))
    return result

In [36]:
dpt, arr, dpt_date = 'GMP','CJU','20170520'
json=crawling_BX_data(dpt,arr,dpt_date)
raw_to_df(json)

Crawling airbusan homepage schedule site
Start Simple crawling :  https://www.airbusan.com/web/bookingApi/domesticAvail
>> Parameters
foc:N , depDate:20170520 , bookingCategory:Individual , depCity:GMP , arrCity:CJU , bookingClass:ES , 
End Simple crawling


,date,flt,start,end,fare1,fare2,tax1,tax2,seat
0,20170520,BX8011,0700,0805,95000,95000,2200,4000,9
1,20170520,BX8013,0720,0830,95000,95000,2200,4000,8
2,20170520,BX8027,0750,0855,95000,95000,2200,4000,2
3,20170520,BX8029,1125,1230,95000,95000,2200,4000,0
4,20170520,BX8021,1445,1555,73600,80000,2200,4000,9
5,20170520,BX8015,1810,1915,48000,80000,2200,4000,9


In [28]:
## 하루치 데이터 읽어오기
dpt, arr, dpt_date = 'GMP','CJU','20170520'
#dpt, arr, dpt_date, dom_int = 'ICN','NRT','2017-05-01','I'

read_BX_1day_fare(dpt,arr,dpt_date)

Crawling airbusan homepage schedule site
Start Simple crawling :  https://www.airbusan.com/web/bookingApi/domesticAvail
>> Parameters
foc:N , depDate:20170520 , bookingCategory:Individual , depCity:GMP , arrCity:CJU , bookingClass:ES , 
End Simple crawling


,date,flt,start,end,fare1,fare2,tax1,tax2,seat
0,20170520,BX8011,0700,0805,95000,95000,2200,4000,9
1,20170520,BX8013,0720,0830,95000,95000,2200,4000,6
2,20170520,BX8027,0750,0855,95000,95000,2200,4000,2
3,20170520,BX8029,1125,1230,95000,95000,2200,4000,0
4,20170520,BX8021,1445,1555,72000,80000,2200,4000,9
5,20170520,BX8015,1810,1915,48000,80000,2200,4000,9
6,20170520,min,max,mean,48000.0,95000.0,73750.0,,


In [32]:
## 정해진 기간의 데이터 읽어오기
dpt, arr, = 'GMP','CJU'
start,end = 0, 31 ## 읽어올 데이터 기간
read_BX_date_range_fare(dpt,arr,start,end)

Crawling airbusan homepage schedule site
Start Simple crawling :  https://www.airbusan.com/web/bookingApi/domesticAvail
>> Parameters
foc:N , depDate:20170508 , bookingCategory:Individual , depCity:GMP , arrCity:CJU , bookingClass:ES , 
End Simple crawling
Crawling airbusan homepage schedule site
Start Simple crawling :  https://www.airbusan.com/web/bookingApi/domesticAvail
>> Parameters
foc:N , depDate:20170509 , bookingCategory:Individual , depCity:GMP , arrCity:CJU , bookingClass:ES , 
End Simple crawling
Crawling airbusan homepage schedule site
Start Simple crawling :  https://www.airbusan.com/web/bookingApi/domesticAvail
>> Parameters
foc:N , depDate:20170510 , bookingCategory:Individual , depCity:GMP , arrCity:CJU , bookingClass:ES , 
End Simple crawling
Crawling airbusan homepage schedule site
Start Simple crawling :  https://www.airbusan.com/web/bookingApi/domesticAvail
>> Parameters
foc:N , depDate:20170511 , bookingCategory:Individual , depCity:GMP , arrCity:CJU , bookingClas

,date,flt,start,end,fare1,fare2,tax1,tax2,seat
0,20170520,BX8011,0700,0805,95000,95000,2200,4000,9
1,20170520,BX8013,0720,0830,95000,95000,2200,4000,6
2,20170520,BX8027,0750,0855,95000,95000,2200,4000,2
3,20170520,BX8029,1125,1230,95000,95000,2200,4000,0
4,20170520,BX8021,1445,1555,72000,80000,2200,4000,9
5,20170520,BX8015,1810,1915,48000,80000,2200,4000,9
6,20170508,min,max,mean,48000.0,95000.0,73750.0,,
7,20170520,BX8011,0700,0805,95000,95000,2200,4000,9
8,20170520,BX8013,0720,0830,95000,95000,2200,4000,6
9,20170520,BX8027,0750,0855,95000,95000,2200,4000,2


In [33]:
## 정해진 기간의 데이터 읽어오기
dpt, arr, = 'GMP','CJU'
start,end = 31, 46 ## 읽어올 데이터 기간
read_BX_date_range_fare(dpt,arr,start,end)

Crawling airbusan homepage schedule site
Start Simple crawling :  https://www.airbusan.com/web/bookingApi/domesticAvail
>> Parameters
foc:N , depDate:20170608 , bookingCategory:Individual , depCity:GMP , arrCity:CJU , bookingClass:ES , 
End Simple crawling
Crawling airbusan homepage schedule site
Start Simple crawling :  https://www.airbusan.com/web/bookingApi/domesticAvail
>> Parameters
foc:N , depDate:20170609 , bookingCategory:Individual , depCity:GMP , arrCity:CJU , bookingClass:ES , 
End Simple crawling
Crawling airbusan homepage schedule site
Start Simple crawling :  https://www.airbusan.com/web/bookingApi/domesticAvail
>> Parameters
foc:N , depDate:20170610 , bookingCategory:Individual , depCity:GMP , arrCity:CJU , bookingClass:ES , 
End Simple crawling
Crawling airbusan homepage schedule site
Start Simple crawling :  https://www.airbusan.com/web/bookingApi/domesticAvail
>> Parameters
foc:N , depDate:20170611 , bookingCategory:Individual , depCity:GMP , arrCity:CJU , bookingClas

,date,flt,start,end,fare1,fare2,tax1,tax2,seat
0,20170520,BX8011,0700,0805,95000,95000,2200,4000,9
1,20170520,BX8013,0720,0830,95000,95000,2200,4000,6
2,20170520,BX8027,0750,0855,95000,95000,2200,4000,2
3,20170520,BX8029,1125,1230,95000,95000,2200,4000,0
4,20170520,BX8021,1445,1555,72000,80000,2200,4000,9
5,20170520,BX8015,1810,1915,48000,80000,2200,4000,9
6,20170608,min,max,mean,48000.0,95000.0,73750.0,,
7,20170520,BX8011,0700,0805,95000,95000,2200,4000,9
8,20170520,BX8013,0720,0830,95000,95000,2200,4000,6
9,20170520,BX8027,0750,0855,95000,95000,2200,4000,2


In [34]:
## 정해진 기간의 데이터 읽어오기
dpt, arr, = 'GMP','CJU'
start,end = 46, 90 ## 읽어올 데이터 기간
read_BX_date_range_fare(dpt,arr,start,end)

Crawling airbusan homepage schedule site
Start Simple crawling :  https://www.airbusan.com/web/bookingApi/domesticAvail
>> Parameters
foc:N , depDate:20170623 , bookingCategory:Individual , depCity:GMP , arrCity:CJU , bookingClass:ES , 
End Simple crawling
Crawling airbusan homepage schedule site
Start Simple crawling :  https://www.airbusan.com/web/bookingApi/domesticAvail
>> Parameters
foc:N , depDate:20170624 , bookingCategory:Individual , depCity:GMP , arrCity:CJU , bookingClass:ES , 
End Simple crawling
Crawling airbusan homepage schedule site
Start Simple crawling :  https://www.airbusan.com/web/bookingApi/domesticAvail
>> Parameters
foc:N , depDate:20170625 , bookingCategory:Individual , depCity:GMP , arrCity:CJU , bookingClass:ES , 
End Simple crawling
Crawling airbusan homepage schedule site
Start Simple crawling :  https://www.airbusan.com/web/bookingApi/domesticAvail
>> Parameters
foc:N , depDate:20170626 , bookingCategory:Individual , depCity:GMP , arrCity:CJU , bookingClas

,date,flt,start,end,fare1,fare2,tax1,tax2,seat
0,20170520,BX8011,0700,0805,95000,95000,2200,4000,9
1,20170520,BX8013,0720,0830,95000,95000,2200,4000,6
2,20170520,BX8027,0750,0855,95000,95000,2200,4000,2
3,20170520,BX8029,1125,1230,95000,95000,2200,4000,0
4,20170520,BX8021,1445,1555,72000,80000,2200,4000,9
5,20170520,BX8015,1810,1915,48000,80000,2200,4000,9
6,20170623,min,max,mean,48000.0,95000.0,73750.0,,
7,20170520,BX8011,0700,0805,95000,95000,2200,4000,9
8,20170520,BX8013,0720,0830,95000,95000,2200,4000,6
9,20170520,BX8027,0750,0855,95000,95000,2200,4000,2
